In [ ]:
from wait_for import wait_for
from collections import namedtuple
from cfme.utils.conf import credentials, cfme_data
from cfme.utils.appliance import IPAppliance
from cfme.test_framework.sprout.client import SproutClient

TimedCommand = namedtuple('TimedCommand', ['command', 'timeout'])

""" The code snips below will allow you to auto configure 1 or more cfme appliance(s)
in single/multiple or dedicated scenarios. Please grab x amount of unconfigured appliances
from sprout that you require, making sure they are all of the same version number. To get
appliances to show in 'My Appliances' on sprout create a credentials.local.yaml in conf/
with the following lines.

sprout:
    username: Myusername
    password: mypassword

NOTE: This setups take 5+ mins to processes, please run 'tail -f log/cfme.log' to follow.
Or Wait until the cells have generated a run number and an out"""

# provider = cfme_data['basic_info']['ha_provider'] # Uncomment this line if you want a HA environment

app_cfme_ver = '5.8.2.0' #Change to specifiy your desired version

sprout_client = SproutClient.from_config()
apps, request_id = sprout_client.provision_appliances(version=str(app_cfme_ver),
    count=3, preconfigured=False, lease_time=360, provider=provider)

print apps

app0_ip = apps[0]
app1_ip = apps[1]
app2_ip = apps[2]
#Comment out the above 6 lines and use the 3 below if you want to use your own appliances

#app0_ip = IPAppliance('10.8.198.132') #1st appliance ip here (Used in single/dedicated)
#app1_ip = IPAppliance('10.8.198.192') #2nd appliance ip here (Used in multi/dedicated)
#app2_ip = IPAppliance('10.8.198.125') #3rd appliance ip here (Used for HA scenario)

dbname = 'vmdb_production' #Change if you require a different databasename

dbuser = 'root' #Change if you require a different username

pwd = credentials['database']['password'] #Change to set password for your database

region = '0' #Change this if you require a different region number

ip0 = app0_ip.address #Used when fetching v2_key and HA nodes
ip2 = app2_ip.address #Used for HA nodes
opt = '5' if app0_ip.version >= "5.8" else '8'
port = (ip0, '') if app0_ip.version >= "5.8" else (ip0,)
rep = '6' if cfme_version >= "5.8" else '9'
mon = '9' if cfme_version >= "5.8" else '12'

In [ ]:
'''Run this cell if you require a single appliance configuration'''

command_set = ('ap', '', opt, '1', '1', 'y', '1', 'n', region, pwd,
                    TimedCommand(pwd, 360), '')

app0_ip.appliance_console.run_commands(command_set)
app0_ip.evmserverd.wait_for_running()
app0_ip.wait_for_miq_ready()


In [ ]:
'''Run this after the first two cells have completed for a multi appliance single region
configuration (distributed setup, 1st appliance has a local database and workers, 2nd
appliance has workers pointing at 1st appliance)'''

command_set = ('ap', '', opt, '2', ip0, '', pwd, '', '3') + port + (dbname, dbuser, pwd,
                    TimedCommand(pwd, 360), '')

app1_ip.appliance_console.run_commands(command_set)
app1_ip.evmserverd.wait_for_running()
app1_ip.wait_for_miq_ready()

In [ ]:
'''Run this after cell 1 for multi appliance single region using dedicated database
(1st appliance database only and 2nd appliance workers only pointing at the dedicated
database)'''

command_set = ('ap', '', opt, '1', '1', 'y', '1', 'y', pwd, TimedCommand(pwd, 360), '')
#Configures dedicated db

app0_ip.appliance_console.run_commands(command_set)
wait_for(lambda: app0_ip.db.is_dedicated_active)

command_set_0 = ('ap', '', opt, '1', '2', region, 'y') + port + (dbname, dbuser, pwd,
                    TimedCommand(pwd, 360), '') #Creates reigion in dedicated db

app1_ip.appliance_console.run_commands(command_set_0)
app1_ip.evmserverd.wait_for_running()
app1_ip.wait_for_miq_ready()


In [ ]:
'''Run this after cell 1+4 have completed to turn your env in to HA configuration consisting
of 2 dedicated database's (1 for failover) and 1 worker appliance pointing at the databases'''

command_set0 = ('ap', '', rep, '1', '1', dbname, dbuser, pwd, pwd, ip0, 'y', '') 
#Configures primary database

app0_ip.appliance_console.run_commands(command_set0)

command_set1 = ('ap', '', rep, '2', '1', '2', dbname, dbuser, pwd, pwd, ip0, ip2, 'y',
                    'y', '') #Configures standby database

app2_ip.appliance_console.run_commands(command_set1)

command_set2 = ('ap', '', mon, '1', '')

app1_ip.appliance_console.run_commands(command_set2)
